# Load Pretrained Model and Tokenizer
Load the pretrained T5 model and tokenizer using the `get_pretrained_model` function.

In [1]:
# Importing Libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='tensorflow')

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_pretrained_model(model_name="google/flan-t5-base"):
    """
    Load the pretrained T5 model and tokenizer.

    Parameters:
    - model_name: str, default "google/flan-t5-base", the name of the pretrained model to load

    Returns:
    - tokenizer: Pretrained tokenizer
    - model: Pretrained T5 model
    """
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
    return tokenizer, model

# Load the pretrained model and tokenizer
model_name = "google/flan-t5-base"
tokenizer, model = get_pretrained_model()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# Preprocess Data
Define a function `preprocess_function` to tokenize the input and output examples, and prepare labels.

In [2]:
# Preprocess Data

def preprocess_function(examples, tokenizer):
    """
    Tokenize the dataset for input and output.

    Parameters:
    - examples: Dictionary with "input" and "output" keys
    - tokenizer: Pretrained tokenizer

    Returns:
    - Dictionary with tokenized inputs and labels
    """
    prefix = ""
    if model_name in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b", "google/flan-t5-base"]:
        prefix = "Translate formal explanation to natural language:"


    inputs = tokenizer(
        [prefix + text for text in examples["input"]],
        padding="max_length", truncation=True, max_length=256
    )
    outputs = tokenizer(
        examples["output"],
        padding="max_length", truncation=True, max_length=256
    )
    inputs["labels"] = outputs["input_ids"]
    inputs["decoder_input_ids"] = outputs["input_ids"]
    
    return inputs

# Prepare Dataset
Convert the raw data into a tokenized dataset using the `prepare_dataset` function.

In [3]:
from datasets import Dataset

def prepare_dataset(raw_data, tokenizer):
    """
    Convert raw data into a tokenized dataset.

    Parameters:
    - raw_data: List of dictionaries with "input" and "output" keys
    - tokenizer: Pretrained tokenizer

    Returns:
    - tokenized_datasets: Tokenized dataset
    """
    dataset = Dataset.from_list(raw_data)
    tokenized_datasets = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    return tokenized_datasets

# Sample dataset
raw_data = [
    # --- Simple Condition Rules ---
    {"input": "C: getKitchenCoffee requires [staffCardAvailable]", "output": "You can get coffee from the kitchen if a staff card is available."},
    {"input": "C: enterOffice requires [securityBadge]", "output": "You need a security badge to enter the office."},
    {"input": "C: accessServerRoom requires [adminClearance, keyCard]", "output": "To access the server room, you must have admin clearance and a key card."},
    {"input": "C: driveCompanyCar requires [validLicense, authorization]", "output": "Driving the company car requires both a valid license and authorization."},
    {"input": "C: openSafe requires [correctPinCode]", "output": "The safe can only be opened with the correct PIN code."},
    {"input": "C: borrowLaptop requires [managerApproval]", "output": "You need manager approval to borrow a laptop."},
    {"input": "C: accessConfidentialFile requires [encryptionKey, clearanceLevel3]", "output": "To access the confidential file, you must have an encryption key and level 3 clearance."},

    # --- Complex Condition Rules ---
    {"input": "C: attendMeeting requires [calendarInvite]", "output": "To attend the meeting, you must have a calendar invite."},
    {"input": "C: accessProjectFolder requires [vpnAccess, teamMembership]", "output": "To access the project folder, you need VPN access and team membership."},
    {"input": "C: printDocuments requires [printerConnected, paperAvailable]", "output": "You can print documents only if the printer is connected and has paper available."},
    {"input": "C: unlockDevice requires [correctPassword, biometricScan]", "output": "You can unlock the device only with the correct password and a biometric scan."},
    {"input": "C: approveExpense requires [budgetApproval, financeSignature]", "output": "An expense can be approved only with budget approval and a finance department signature."},
    {"input": "C: deploySoftware requires [codeReview, securityCheck]", "output": "Software can be deployed only after a code review and a security check."},
    {"input": "C: bookFlight requires [travelApproval, availableBudget]", "output": "Booking a flight requires travel approval and an available budget."},
    {"input": "C: purchaseItem requires [sufficientBalance]", "output": "You can purchase the item only if you have sufficient balance."},
    {"input": "C: leaveBuilding requires [exitPass]", "output": "To leave the building, you need an exit pass."},
    {"input": "C: sendEmail requires [internetConnection]", "output": "You need an internet connection to send an email."},
    {"input": "C: startProject requires [projectApproval, assignedTeam]", "output": "A project can only start with approval and an assigned team."},

    # --- Simple Preference Rules ---
    {"input": "V: getOwnCard (0,0,0) > getOthersCard (0,0,2)", "output": "It is preferable to get your own card rather than using someone else's."},
    {"input": "V: useCompanyLaptop (1,1,1) > usePersonalLaptop (0,0,1)", "output": "Using a company laptop is preferred over using a personal laptop."},
    {"input": "V: takeElevator (3,0,1) > takeStairs (2,1,2)", "output": "Taking the elevator is preferred over taking the stairs in this situation."},
    {"input": "V: buyNewPrinter (5,2,1) > repairOldPrinter (3,3,3)", "output": "Buying a new printer is preferable to repairing the old one."},
    {"input": "V: rentCar (4,0,2) > usePublicTransport (3,1,4)", "output": "Renting a car is a better choice than using public transportation."},
    {"input": "V: installSoftwareUpdate (2,0,1) > postponeUpdate (1,1,3)", "output": "Installing the software update now is better than postponing it."},

    # --- Complex Preference Rules ---
    {"input": "V: attendMeetingRemotely (1,1,0) > attendInPerson (0,0,2)", "output": "It is preferable to attend the meeting remotely rather than in person."},
    {"input": "V: useCloudStorage (2,0,0) > useLocalStorage (1,1,3)", "output": "Using cloud storage is preferable to using local storage."},
    {"input": "V: printInBlackWhite (0,0,0) > printInColor (2,0,3)", "output": "Printing in black and white is preferred over printing in color."},
    {"input": "V: enableTwoFactorAuth (3,0,1) > relyOnPasswordOnly (1,1,3)", "output": "Enabling two-factor authentication is better than relying only on a password."},
    {"input": "V: hireExperiencedCandidate (5,1,0) > hireNewGraduate (3,2,4)", "output": "Hiring an experienced candidate is preferable to hiring a new graduate."},
    {"input": "V: deployStableVersion (2,0,0) > deployBetaVersion (1,1,3)", "output": "Deploying the stable version is preferable to deploying the beta version."},

    # --- Real-world Inspired Condition Rules ---
    {"input": "C: applyForLoan requires [creditScoreAbove600, steadyIncome]", "output": "To apply for a loan, you must have a credit score above 600 and a steady income."},
    {"input": "C: registerForExam requires [courseEnrollment, feePaid]", "output": "You can register for the exam only if you are enrolled in the course and have paid the fee."},
    {"input": "C: enterGym requires [membershipCard]", "output": "You need a membership card to enter the gym."},
    {"input": "C: submitAssignment requires [beforeDeadline, correctFormat]", "output": "Assignments must be submitted before the deadline and in the correct format."},
    {"input": "C: boardFlight requires [passport, boardingPass]", "output": "You must have a passport and boarding pass to board the flight."},
    {"input": "C: withdrawCash requires [debitCard, sufficientBalance]", "output": "You can withdraw cash only if you have a debit card and sufficient balance."},

    # --- Real-world Inspired Preference Rules ---
    {"input": "V: workFromHome (4,1,0) > workFromOffice (3,2,2)", "output": "Working from home is preferable to working from the office in this situation."},
    {"input": "V: useEmailForOfficialCommunication (3,0,1) > useMessagingApps (1,1,2)", "output": "Using email for official communication is better than using messaging apps."},
    {"input": "V: driveElectricCar (5,0,1) > drivePetrolCar (3,2,3)", "output": "Driving an electric car is preferable to driving a petrol car."},
    {"input": "V: buyOrganicFood (4,1,0) > buyNonOrganicFood (3,2,2)", "output": "Buying organic food is preferable to buying non-organic food."},
    {"input": "V: useRecyclableMaterials (3,0,0) > usePlasticMaterials (1,2,4)", "output": "Using recyclable materials is preferred over using plastic materials."},

    # --- Additional Simple Condition Rules ---
    {"input": "C: useConferenceRoom requires [booking, departmentApproval]", "output": "To use the conference room, you need a booking and department approval."},
    {"input": "C: accessDatabase requires [loginCredentials, securityTraining]", "output": "You need login credentials and completed security training to access the database."},
    {"input": "C: submitExpenseReport requires [receipts, managerSignoff]", "output": "Submitting an expense report requires receipts and manager sign-off."},
    {"input": "C: requestVacation requires [availablePTO, supervisorApproval]", "output": "To request vacation, you must have available PTO and supervisor approval."},
    {"input": "C: useCompanyVPN requires [secureConnection, validUserAccount]", "output": "Using the company VPN requires a secure connection and a valid user account."},
    {"input": "C: enterRestrictedArea requires [securityClearance, escortPresent]", "output": "Entering the restricted area requires security clearance and an escort present."},
    {"input": "C: retrieveBackupData requires [administratorRights, recoveryKey]", "output": "To retrieve backup data, you need administrator rights and a recovery key."},
    {"input": "C: modifyCustomerRecords requires [supervisorPermission, auditTrail]", "output": "Modifying customer records requires supervisor permission and creates an audit trail."},
    {"input": "C: installNewSoftware requires [ITapproval, compatibilityCheck]", "output": "Installing new software requires IT approval and a compatibility check."},
    {"input": "C: accessBuildingAfterHours requires [specialAuthorization, securityNotification]", "output": "Accessing the building after hours requires special authorization and security notification."},

    # --- Additional Complex Condition Rules ---
    {"input": "C: processClientPayment requires [verifiedAccount, paymentAuthorization]", "output": "Processing a client payment requires a verified account and payment authorization."},
    {"input": "C: releaseProductVersion requires [qualityAssurance, stakeholderSignoff]", "output": "Releasing a product version requires quality assurance and stakeholder sign-off."},
    {"input": "C: initiateEmergencyProtocol requires [securityBreach, managerAuthorization]", "output": "The emergency protocol can only be initiated during a security breach with manager authorization."},
    {"input": "C: scheduleClientMeeting requires [availableTimeSlot, clientConfirmation]", "output": "Scheduling a client meeting requires an available time slot and client confirmation."},
    {"input": "C: accessFinancialRecords requires [financeDepartmentRole, auditPurpose]", "output": "Accessing financial records requires having a finance department role and an audit purpose."},
    {"input": "C: performSystemMaintenance requires [scheduledDowntime, userNotification]", "output": "System maintenance can be performed only during scheduled downtime after user notification."},
    {"input": "C: issueRefund requires [originalReceipt, managerApproval]", "output": "A refund can be issued only with the original receipt and manager approval."},
    {"input": "C: updateFirmwareRemotely requires [secureChannel, deviceOnline]", "output": "Updating firmware remotely requires a secure channel and the device being online."},
    {"input": "C: transferSensitiveData requires [encryptedConnection, dataSanitization]", "output": "Transferring sensitive data requires an encrypted connection and data sanitization."},
    {"input": "C: createNewUserAccount requires [identityVerification, roleAssignment]", "output": "Creating a new user account requires identity verification and role assignment."},

    # --- Additional Simple Preference Rules ---
    {"input": "V: submitDigitalForm (2,0,0) > submitPaperForm (1,1,3)", "output": "Submitting a digital form is preferred over submitting a paper form."},
    {"input": "V: useVideoConference (3,0,1) > useTeleconference (2,1,2)", "output": "Using video conferencing is preferred over using teleconferencing."},
    {"input": "V: adoptAutomatedSystem (4,1,0) > maintainManualProcess (2,2,3)", "output": "Adopting an automated system is preferable to maintaining a manual process."},
    {"input": "V: useEnvironmentallyFriendlyOption (3,0,0) > useStandardOption (1,1,3)", "output": "Using the environmentally friendly option is preferred over using the standard option."},
    {"input": "V: implementPreventativeMeasures (4,0,1) > reactToIssues (2,1,3)", "output": "Implementing preventative measures is better than reacting to issues after they occur."},
    {"input": "V: followStandardProcedure (3,0,1) > createCustomSolution (2,2,3)", "output": "Following standard procedure is preferred over creating a custom solution."},
    {"input": "V: purchaseWarrantyExtension (2,0,1) > skipWarranty (1,1,3)", "output": "Purchasing a warranty extension is preferable to skipping warranty coverage."},
    {"input": "V: useExistingTemplate (2,0,0) > createNewTemplate (1,2,2)", "output": "Using an existing template is preferred over creating a new template."},
    {"input": "V: scheduleRegularMaintenance (3,0,0) > repairWhenBroken (1,2,4)", "output": "Scheduling regular maintenance is preferable to repairing when broken."},
    {"input": "V: followUpInWriting (2,0,0) > verbalFollowUpOnly (1,1,3)", "output": "Following up in writing is better than only following up verbally."},

    # --- Additional Complex Preference Rules ---
    {"input": "V: implementCloudBackup (4,0,1) > relyOnLocalBackup (2,1,3)", "output": "Implementing cloud backup is preferable to relying solely on local backup."},
    {"input": "V: hireInternalCandidate (3,0,1) > recruitExternally (2,2,3)", "output": "Hiring an internal candidate is preferable to recruiting externally."},
    {"input": "V: useSingleSignOn (3,0,0) > maintainMultipleLogins (1,1,4)", "output": "Using single sign-on is better than maintaining multiple logins."},
    {"input": "V: conductVirtualTraining (3,0,1) > requireInPersonTraining (2,2,3)", "output": "Conducting virtual training is preferable to requiring in-person training."},
    {"input": "V: developCustomizedSolution (4,2,1) > purchaseOffTheShelf (3,1,2)", "output": "Developing a customized solution is preferred over purchasing an off-the-shelf product."},
    {"input": "V: implementAgileMethodology (4,1,1) > useWaterfallApproach (2,2,3)", "output": "Implementing agile methodology is preferred over using a waterfall approach."},
    {"input": "V: negotiateMultiYearContract (3,1,0) > renewAnnually (2,2,3)", "output": "Negotiating a multi-year contract is preferable to renewing annually."},
    {"input": "V: outsourceNonCoreFunction (3,0,1) > maintainInHouse (2,2,3)", "output": "Outsourcing non-core functions is preferred over maintaining them in-house."},
    {"input": "V: investInEmployeeTraining (4,1,0) > hireNewExpertise (3,2,3)", "output": "Investing in employee training is preferable to hiring new expertise."},
    {"input": "V: standardizeEquipment (3,0,1) > allowVariedDevices (2,2,3)", "output": "Standardizing equipment is better than allowing varied devices."},

    # --- Additional Real-world Inspired Condition Rules ---
    {"input": "C: checkOutLibraryBook requires [validLibraryCard, noOverdueItems]", "output": "To check out a library book, you need a valid library card and no overdue items."},
    {"input": "C: receivePrescription requires [doctorConsultation, validIdentification]", "output": "Receiving a prescription requires a doctor consultation and valid identification."},
    {"input": "C: enrollInHealthInsurance requires [eligibilityDocuments, enrollmentPeriod]", "output": "To enroll in health insurance, you need eligibility documents during the enrollment period."},
    {"input": "C: registerVehicle requires [proofOfOwnership, insuranceCertificate]", "output": "Registering a vehicle requires proof of ownership and an insurance certificate."},
    {"input": "C: openBankAccount requires [identityProof, addressVerification]", "output": "Opening a bank account requires identity proof and address verification."},
    {"input": "C: applyForPassport requires [birthCertificate, photographID]", "output": "Applying for a passport requires a birth certificate and photograph ID."},
    {"input": "C: rentApartment requires [creditCheck, depositPayment]", "output": "Renting an apartment requires passing a credit check and paying a deposit."},
    {"input": "C: enrollInUniversity requires [previousEducationRecords, entranceExamScore]", "output": "Enrolling in a university requires previous education records and an entrance exam score."},
    {"input": "C: purchaseAlcohol requires [legalAge, validID]", "output": "Purchasing alcohol requires being of legal age and having valid ID."},
    {"input": "C: applyForJobPosition requires [resume, coverLetter]", "output": "Applying for a job position requires a resume and cover letter."},

    # --- Additional Real-world Inspired Preference Rules ---
    {"input": "V: useRenewableEnergy (5,0,0) > useFossilFuels (2,2,4)", "output": "Using renewable energy is preferable to using fossil fuels."},
    {"input": "V: supportLocalBusiness (4,0,1) > purchaseFromChains (2,1,3)", "output": "Supporting local businesses is better than purchasing from large chains."},
    {"input": "V: choosePublicTransportation (4,0,0) > drivePrivateVehicle (2,2,3)", "output": "Choosing public transportation is preferable to driving a private vehicle."},
    {"input": "V: adoptPaperlessProcess (3,0,0) > usePhysicalDocuments (1,2,3)", "output": "Adopting a paperless process is better than using physical documents."},
    {"input": "V: purchaseEnergyEfficientAppliance (4,1,0) > buyStandardAppliance (2,2,3)", "output": "Purchasing an energy-efficient appliance is preferable to buying a standard appliance."},
    {"input": "V: eatBalancedMeal (4,0,0) > consumeProcessedFood (1,2,4)", "output": "Eating a balanced meal is preferable to consuming processed food."},
    {"input": "V: exerciseRegularly (4,0,0) > remainSedentary (1,2,5)", "output": "Exercising regularly is better than remaining sedentary."},
    {"input": "V: maintainWorkLifeBalance (4,0,0) > workExcessiveHours (1,3,4)", "output": "Maintaining work-life balance is preferable to working excessive hours."},
    {"input": "V: saveForRetirement (4,0,0) > spendEntireIncome (1,2,5)", "output": "Saving for retirement is better than spending your entire income."},
    {"input": "V: learnNewSkills (4,0,0) > stagnateCareerGrowth (1,2,4)", "output": "Learning new skills is preferable to stagnating in career growth."},

    # --- Edge Case Condition Rules ---
    {"input": "C: accessEmergencyExit requires []", "output": "Anyone can access the emergency exit without restrictions."},
    {"input": "C: accessPremiumFeatures requires [paidSubscription, verifiedAccount, completedTutorial]", "output": "Accessing premium features requires a paid subscription, verified account, and completed tutorial."},
    {"input": "C: launchNuclearProtocol requires [presidentialAuthorization, secondaryVerification, emergencySituation, securityClearanceLevel5]", "output": "Launching the nuclear protocol requires presidential authorization, secondary verification, an emergency situation, and level 5 security clearance."},
    {"input": "C: viewPublicWebsite requires []", "output": "Anyone can view the public website without restrictions."},
    {"input": "C: useRestroom requires []", "output": "Anyone can use the restroom without specific requirements."},

    # --- Edge Case Preference Rules ---
    {"input": "V: doNothing (0,0,0) > takeAction (0,0,0)", "output": "In this case, doing nothing is exactly equivalent to taking action."},
    {"input": "V: optionA (10,0,0) > optionB (0,10,10)", "output": "Option A is significantly preferred over Option B."},
    {"input": "V: quickFix (1,0,9) > properSolution (9,9,1)", "output": "A quick fix, despite its drawbacks, is preferred over a proper solution in this context."},
    {"input": "V: saveTime (5,5,5) > saveMoney (5,5,5)", "output": "Saving time and saving money are equally valuable in this situation."},
    {"input": "V: maintainStatus (0,0,0) > attemptImprovement (10,10,10)", "output": "Maintaining the status quo is preferable to attempting improvement despite the potential benefits."},

    # --- Industry-Specific Condition Rules ---
    {"input": "C: prescribeMedication requires [medicalLicense, patientConsultation, medicalNecessity]", "output": "Prescribing medication requires a medical license, patient consultation, and medical necessity."},
    {"input": "C: fileCourtMotion requires [barAdmission, clientConsent, validCaseNumber]", "output": "Filing a court motion requires bar admission, client consent, and a valid case number."},
    {"input": "C: conductFinancialAudit requires [accountingCertification, companyAccess, auditScope]", "output": "Conducting a financial audit requires accounting certification, company access, and a defined audit scope."},
    {"input": "C: deployMilitaryPersonnel requires [commandOrder, threatAssessment, resourceAvailability]", "output": "Deploying military personnel requires a command order, threat assessment, and resource availability."},
    {"input": "C: approveConstructionPermit requires [engineeringPlans, safetyInspection, zoneCompliance]", "output": "Approving a construction permit requires engineering plans, a safety inspection, and zone compliance."},
    {"input": "C: conductScientificExperiment requires [ethicsApproval, methodologyValidation, safetyProtocols]", "output": "Conducting a scientific experiment requires ethics approval, methodology validation, and safety protocols."},
    {"input": "C: authorizeInsuranceClaim requires [policyVerification, incidentDocumentation, coverageEligibility]", "output": "Authorizing an insurance claim requires policy verification, incident documentation, and coverage eligibility."},
    {"input": "C: performSurgery requires [surgicalLicense, patientConsent, medicalNecessity, operatingRoomAvailability]", "output": "Performing surgery requires a surgical license, patient consent, medical necessity, and operating room availability."},
    {"input": "C: approveEducationalCurriculum requires [accreditationStandards, facultyEndorsement, administrativeReview]", "output": "Approving an educational curriculum requires meeting accreditation standards, faculty endorsement, and administrative review."},
    {"input": "C: launchMarketingCampaign requires [brandGuidelines, budgetApproval, targetAudienceResearch]", "output": "Launching a marketing campaign requires following brand guidelines, budget approval, and target audience research."},

    # --- Industry-Specific Preference Rules ---
    {"input": "V: useGenericMedication (4,0,0) > prescribeBrandName (2,3,2)", "output": "Using generic medication is preferred over prescribing brand-name drugs."},
    {"input": "V: settleCaseOutOfCourt (4,1,1) > proceedToTrial (2,3,4)", "output": "Settling a case out of court is preferable to proceeding to trial."},
    {"input": "V: investInIndexFund (3,0,1) > pickIndividualStocks (2,3,4)", "output": "Investing in an index fund is preferred over picking individual stocks."},
    {"input": "V: implementAgileDevelopment (4,1,0) > useTraditionalWaterfall (2,2,3)", "output": "Implementing agile development is preferred over using traditional waterfall methodology."},
    {"input": "V: employPreventiveMaintenance (4,0,0) > reactiveRepairs (1,2,4)", "output": "Employing preventive maintenance is better than relying on reactive repairs."},
    {"input": "V: utilizeRenewableMaterials (4,0,0) > useTraditionalMaterials (2,2,3)", "output": "Utilizing renewable materials is preferable to using traditional materials."},
    {"input": "V: adoptEvidenceBasedPractices (4,0,0) > followTradition (2,2,3)", "output": "Adopting evidence-based practices is better than simply following tradition."},
    {"input": "V: implementDistributedSystem (4,1,1) > buildMonolithicSystem (2,2,3)", "output": "Implementing a distributed system is preferable to building a monolithic system."},
    {"input": "V: offerFlexibleWorkArrangements (4,0,0) > requireFixedSchedules (2,2,3)", "output": "Offering flexible work arrangements is preferred over requiring fixed schedules."},
    {"input": "V: useNaturalIngredients (4,0,0) > useArtificialAdditives (1,2,4)", "output": "Using natural ingredients is preferable to using artificial additives."},

    # --- Scenario-Based Condition Rules ---
    {"input": "C: evacuateBuilding requires [fireAlarm, securityAnnouncement]", "output": "Evacuating the building requires a fire alarm and security announcement."},
    {"input": "C: activateBackupPower requires [powerOutage, systemFailure]", "output": "Activating backup power requires a power outage and system failure."},
    {"input": "C: initiateDisasterRecovery requires [dataLoss, businessContinuityPlan]", "output": "Initiating disaster recovery requires data loss and a business continuity plan."},
    {"input": "C: quarantineFacility requires [contaminationDetection, healthAuthoritiesNotification]", "output": "Quarantining a facility requires contamination detection and health authorities notification."},
    {"input": "C: implementLockdown requires [securityThreat, emergencyProtocol]", "output": "Implementing a lockdown requires a security threat and emergency protocol."},
    {"input": "C: declareFinancialEmergency requires [budgetDeficit, boardResolution]", "output": "Declaring a financial emergency requires a budget deficit and board resolution."},
    {"input": "C: cancelFlightOperations requires [severeWeather, safetyRiskAssessment]", "output": "Canceling flight operations requires severe weather and a safety risk assessment."},
    {"input": "C: recallProduct requires [qualityDefect, consumerSafetyRisk]", "output": "Recalling a product requires a quality defect and consumer safety risk."},
    {"input": "C: closeTemporarily requires [facilitySafetyConcern, remediationPlan]", "output": "Closing temporarily requires a facility safety concern and a remediation plan."},
    {"input": "C: suspendEmployee requires [policyViolation, humanResourcesInvestigation]", "output": "Suspending an employee requires a policy violation and a human resources investigation."},

    # --- Scenario-Based Preference Rules ---
    {"input": "V: evacuateSafely (5,0,0) > protectProperty (1,2,5)", "output": "Evacuating safely is significantly preferred over protecting property."},
    {"input": "V: prioritizeSafety (5,0,0) > maintainProductivity (1,3,5)", "output": "Prioritizing safety is significantly preferred over maintaining productivity."},
    {"input": "V: communicateTransparently (4,0,0) > witholdInformation (1,3,5)", "output": "Communicating transparently is preferred over withholding information."},
    {"input": "V: addressRootCause (4,0,0) > treatSymptoms (1,2,4)", "output": "Addressing the root cause is preferred over treating only the symptoms."},
    {"input": "V: preserveEvidence (4,0,0) > cleanUpQuickly (1,3,4)", "output": "Preserving evidence is preferred over cleaning up quickly."},
    {"input": "V: followEstablishedProcedures (4,0,0) > improviseSolutions (2,2,3)", "output": "Following established procedures is preferred over improvising solutions."},
    {"input": "V: seekExpertAssistance (4,0,0) > handleInternally (2,2,3)", "output": "Seeking expert assistance is preferred over handling the situation internally."},
    {"input": "V: notifyAffectedParties (4,0,0) > limitInformationSharing (1,3,4)", "output": "Notifying affected parties is preferred over limiting information sharing."},
    {"input": "V: implementTemporarySolution (3,0,1) > waitForPerfectSolution (1,3,5)", "output": "Implementing a temporary solution is preferred over waiting for a perfect solution."},
    {"input": "V: documentIncident (4,0,0) > relyOnMemory (1,2,4)", "output": "Documenting the incident is preferred over relying on memory."},
    # --- Technical/IT Environment Rules ---
    {"input": "C: mergeCodeBranch requires [peerReview, unitTestsPassing, integrationTestsPassing]", "output": "Merging a code branch requires peer review and both unit and integration tests passing."},
    {"input": "C: resetUserPassword requires [identityVerification, securityQuestion, managerApproval]", "output": "Resetting a user password requires identity verification, answering a security question, and manager approval."},
    {"input": "C: configureFirewall requires [networkAdminRights, changeManagementTicket, securityReview]", "output": "Configuring the firewall requires network admin rights, a change management ticket, and security review."},
    {"input": "C: installCriticalUpdate requires [backupCompleted, maintenanceWindow, userNotification]", "output": "Installing a critical update requires completed backup, being within the maintenance window, and user notification."},
    {"input": "V: useContainerization (4,0,1) > manualDeployment (2,2,4)", "output": "Using containerization is preferable to manual deployment."},
    {"input": "V: implementMicroservices (4,1,1) > buildMonolithicApplication (2,2,3)", "output": "Implementing microservices is preferable to building a monolithic application."},
    {"input": "V: adoptDevOpsMethodology (4,0,0) > separateDevelopmentOperations (2,2,4)", "output": "Adopting DevOps methodology is preferable to keeping development and operations separate."},
    {"input": "V: useVersionControl (5,0,0) > manageFilesManually (1,3,5)", "output": "Using version control is significantly preferred over managing files manually."},
    
    # --- Educational Context Rules ---
    {"input": "C: graduateProgram requires [completedRequiredCourses, minimumGPA, thesisDefense]", "output": "Graduating from the program requires completing required courses, maintaining a minimum GPA, and defending a thesis."},
    {"input": "C: teachCourse requires [subjectCertification, pedagogicalTraining, departmentalApproval]", "output": "Teaching a course requires subject certification, pedagogical training, and departmental approval."},
    {"input": "C: submitResearchForPublication requires [originalContent, peerReview, ethicalClearance]", "output": "Submitting research for publication requires original content, peer review, and ethical clearance."},
    {"input": "C: accessStudentRecords requires [administrativePermission, educationalPurpose, dataPrivacyTraining]", "output": "Accessing student records requires administrative permission, an educational purpose, and data privacy training."},
    {"input": "V: provideConstructiveFeedback (4,0,0) > giveGradeOnly (2,1,3)", "output": "Providing constructive feedback is preferable to giving only a grade."},
    {"input": "V: accommodateLearningDifferences (4,0,0) > enforceStandardApproach (2,2,3)", "output": "Accommodating learning differences is preferable to enforcing a standard approach for all students."},
    {"input": "V: encourageActiveParticipation (4,0,0) > lectureExclusively (2,2,3)", "output": "Encouraging active participation is preferable to exclusively lecturing."},
    {"input": "V: assessComprehensively (4,0,0) > relyOnFinalExamOnly (2,2,4)", "output": "Assessing comprehensively throughout the course is preferable to relying only on a final exam."},

]

# Prepare dataset
tokenized_datasets = prepare_dataset(raw_data, tokenizer)

# Split the dataset into training and evaluation sets
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

In [4]:
# Print dataset statistics
print(f"Total dataset size: {len(tokenized_datasets)}")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Peek into the datasets
print("\nSample from training dataset:")
print(train_dataset[0])

print("\nSample from evaluation dataset:")
print(eval_dataset[0])

Total dataset size: 167
Training dataset size: 133
Evaluation dataset size: 34

Sample from training dataset:
{'input': 'C: getKitchenCoffee requires [staffCardAvailable]', 'output': 'You can get coffee from the kitchen if a staff card is available.', 'input_ids': [30355, 15, 4727, 7295, 12, 793, 1612, 10, 254, 10, 129, 439, 7059, 35, 3881, 7398, 15, 2311, 784, 26416, 6936, 26, 188, 900, 173, 179, 908, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
test_text = "C: getKitchenCoffee requires [staffCardAvailable]"
tokenized = tokenizer(test_text, return_tensors="pt")

print("Tokenized Input:", tokenized)
decoded = tokenizer.decode(tokenized["input_ids"][0])
print("Decoded Back:", decoded)

Tokenized Input: {'input_ids': tensor([[  205,    10,   129,   439,  7059,    35,  3881,  7398,    15,  2311,
           784, 26416,  6936,    26,   188,   900,   173,   179,   908,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Decoded Back: C: getKitchenCoffee requires [staffCardAvailable]</s>


# Train Model
Fine-tune the pretrained T5 model using the tokenized dataset with the `train_model` function.

In [6]:
from transformers import TrainingArguments, Trainer

def train_model(train_dataset, eval_dataset, model):
    """Fine-tune the T5 model and ensure loss is decreasing."""
    training_args = TrainingArguments(
        output_dir="./t5_nle",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,  # Start lower to check progress
        # learning_rate=1,  # Slightly higher than before
        # weight_decay=0.01,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=50,  # Evaluate more often
        save_steps=100,
        logging_steps=10,  # Log loss every 10 steps
        logging_dir="./logs",
        report_to="none",
        do_eval=True,
        load_best_model_at_end=True,  # Ensures best validation model is used
        metric_for_best_model="loss",
        label_smoothing_factor=0.1
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )

    print("Starting training...")
    trainer.train()
    print("Training complete.")

    return model


# Train the model
trained_model = train_model(train_dataset, eval_dataset, model)

c:\Users\bar24\anaconda3\envs\ExplainableAI\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,7.330500,7.040112


Training complete.


## Save the trained model

In [8]:
import torch
import pickle

def save_model(model, tokenizer, model_path="trained_model.pth", tokenizer_path="tokenizer.pkl"):
    """
    Save the trained model and tokenizer.

    Parameters:
    - model: Trained T5 model
    - tokenizer: Tokenizer corresponding to the model
    - model_path: Path to save the model (default: "trained_model.pth")
    - tokenizer_path: Path to save the tokenizer (default: "tokenizer.pkl")
    """
    # Save the model
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    # Save the tokenizer
    with open(tokenizer_path, "wb") as f:
        pickle.dump(tokenizer, f)
    print(f"Tokenizer saved to {tokenizer_path}")

# Save the trained model and tokenizer
save_model(trained_model, tokenizer)

Model saved to trained_model.pth
Tokenizer saved to tokenizer.pkl


# Generate Explanation
Define a function `generate_explanation` to generate natural language explanations using the trained T5 model.

In [7]:
# Generate Explanation

def generate_explanation(model, tokenizer, input_text):
    """Generate a natural explanation using the trained T5 model."""
    input_text = "translate formal explanation to natural language: " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    print("Tokenized input:", inputs["input_ids"])  # Debugging

    output_ids = model.generate(
        **inputs,
        max_length=500,
        num_beams=200,  # Improve quality with beam search
        early_stopping=True
    )

    print("Raw output (token IDs):", output_ids)  # Debugging

    decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return decoded_text


# Example usage
input_text = "C: getKitchenCoffee requires [staffCardAvailable]"
generated_explanation = generate_explanation(trained_model, tokenizer, input_text)
print(f"Input: {input_text}\nGenerated Explanation: {generated_explanation}")

Tokenized input: tensor([[13959,  4727,  7295,    12,   793,  1612,    10,   205,    10,   129,
           439,  7059,    35,  3881,  7398,    15,  2311,   784, 26416,  6936,
            26,   188,   900,   173,   179,   908,     1]], device='cuda:0')
Raw output (token IDs): tensor([[    0,   205,    10,   129,   439,  7059,    35,  3881,  7398,    15,
          2311,   784, 26416,   188,   900,   173,   179,   908,     1]],
       device='cuda:0')
Input: C: getKitchenCoffee requires [staffCardAvailable]
Generated Explanation: C: getKitchenCoffee requires [staffAvailable]


# Test Model
Define a function `test_model` to test the model on a list of inputs and print the generated explanations.

In [ ]:
def test_model(model, tokenizer, test_inputs):
    """
    Test the model on a list of inputs.

    Parameters:
    - model: Trained T5 model
    - tokenizer: Tokenizer corresponding to the model
    - test_inputs: List of formal explanations

    Prints:
    - Generated explanations
    """
    for test_input in test_inputs:
        explanation = generate_explanation(model, tokenizer, test_input)
        print(f"Input: {test_input}\nGenerated Explanation: {explanation}\n")

# Test the model
test_inputs = [
    "C: getKitchenCoffee requires [staffCardAvailable]",
    "V: getOwnCard (0,0,0) > getOthersCard (0,0,2)"
]
test_model(trained_model, tokenizer, test_inputs)

# Example Usage
Demonstrate the usage of the functions with sample data, including loading the model, preparing the dataset, training the model, and testing it with example inputs.

In [ ]:
# Example Usage

# Load the pretrained model and tokenizer
tokenizer, model = get_pretrained_model()

# Sample dataset
raw_data = [
    {
        "input": "C: getKitchenCoffee requires [staffCardAvailable]",
        "output": "You can get coffee from the kitchen if a staff card is available."
    },
    {
        "input": "V: getOwnCard (0,0,0) > getOthersCard (0,0,2)",
        "output": "It is preferable to get your own card rather than using someone else's."
    }
]

# Prepare dataset
tokenized_datasets = prepare_dataset(raw_data, tokenizer)

# Split the dataset into training and evaluation sets
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Train the model
trained_model = train_model(train_dataset, eval_dataset, model)

# Test the model
test_inputs = [
    "C: getKitchenCoffee requires [staffCardAvailable]",
    "V: getOwnCard (0,0,0) > getOthersCard (0,0,2)"
]
test_model(trained_model, tokenizer, test_inputs)